In [1]:
# Parameter estimation using GWBench (numerical derivatives) (ref. AISS)

import numpy as np
# from gwbench import network

import gwbench.basic_functions as bfs
import gwbench.fisher_analysis_tools as fat

# import snr_models as model
# import waveform_lal as wf_lal
# import numpy.linalg as LA

PI = np.pi

In [ ]:
import gwbench.basic_constants as bc

In [ ]:
# Injection parameter calculation

# e0 = 0.0
chi1z = 0.0
chi2z = 0.0

tc = 0.0
phic = 0.0

M1_solar = 1.4    # Change the mass of the binary
M2_solar = 1.4

M_solar = M1_solar + M2_solar          # Total mass in solar mass unit
M_SI = M_solar*bc.Msun                 # Total mass in SI unit
M_sec = M_SI*bc.GNewton/bc.cLight**3   # Total mass in seconds

eta = (M1_solar*M2_solar/(M_solar)**2) 

Mc_solar = M_solar*eta**(3/5)          # Chirp mass in solar mass unit
Mc = M_sec*eta**(3/5)                  # Chirp mass in seconds

In [ ]:
import aiss_model_np as aiss_np     # loading the PSD and can be change from the script
from scipy.integrate import quad    # 

In [ ]:
# Choose the PSD and the desired frequency range

# Fs = 4096


fs = 20
flso = (6.**(3/2) * PI * M_sec)**(-1)

# deltaF = 2**(-2.9)
# f = np.arange(0, Fs, deltaF)

# i_fs = int((fs-0)/deltaF)
# i_flso = int((flso-0)/deltaF) + 1

# f = f[i_fs:i_flso]

f = np.arange(fs, flso, 1)

psd_func = aiss_np.Sh_aLIGO 
psd = psd_func(f)

# Calculating Amplitude corresponding to SNR 10
rho0  = 10

def integrand(f1):
    return f1**(-7/3) / psd_func(f1)

ans, err = quad(integrand, f[0], flso) # quad returns the answer of the quadrature sum and the error, the error is quite high
A =  (rho0**2 / (4*ans))**0.5

DL = ((5./24.)**0.5/PI**(2./3.))*(Mc**(5./6.)/A) # From AISS
DL_Mpc = DL*bc.cLight/bc.Mpc


# print(DL)
# print(A)
# print(flso)
# print(f[-1])
# print(f)

In [ ]:
from network_check import Network # "Network" is a class
import gwbench.wf_class as wfc 

In [ ]:
# choose the desired detectors
# network_spec = []
# initialize the network
# net1 = network.Network()


net1 = Network()
net1.wf = wfc.Waveform()

In [ ]:
inj_params1 = {
    'Mc':    Mc_solar,
    'eta':   0.2499,
    'chi1z': chi1z,
    'chi2z': chi2z,
    'DL':    DL_Mpc,
    'tc':    tc,
    'phic':  phic,
    'iota':  0,
    'ra':    0,
    'dec':   0.0,
    'psi':   0,
    'gmst0': 0
    }

In [ ]:
# variables for which derivatives to be calculated
# deriv_symbs_string = 'tc phic M eta chi1z chi2z'
deriv_symbs_string = 'Mc eta tc phic'


# convert to log or cos
conv_cos = ()
conv_log = ('Mc','eta')

# Earth's rotation
use_rot = 0

# set variables for network
net1.set_net_vars(
    f=f, inj_params=inj_params1,
    deriv_symbs_string=deriv_symbs_string,
    conv_cos=conv_cos, conv_log=conv_log,
    use_rot=use_rot
    )

# choose waveform

# wf_model_name = 'tf2'
# net.set_wf_vars(wf_model_name=wf_model_name)

net.wf = wfc.Waveform()
net.wf.wf_symbs_string = wf_lal.wf_symbs_string
net.wf.hfpc_np = wf_lal.hf
net.wf.hfpc_sp = None

In [ ]:
# import wf_models.tf2_ecc_np as tecc_np
# import wf_models.tf2_ecc_sp as tecc_sp

# wf_model_name = 'tecc'

import wf_models.tf2_2_np as tf2_2_np
import wf_models.tf2_2_sp as tf2_2_sp

In [ ]:
# def select_wf_model_quants(self):
#     np_mod = tecc_np
#     sp_mod = tecc_sp 
    
#     if sp_mod is None: sp_tmp = None
#     else:              sp_tmp = sp_mod.hfpc
        
#     return np_mod.wf_symbs_string, np_mod.hfpc, sp_tmp

# net1.wf.wf_symbs_string, net1.wf.hfpc_np, net1.wf.hfpc_sp = select_wf_model_quants(net1)


def select_wf_model_quants(self):
    np_mod = tf2_2_np
    sp_mod = tf2_2_sp 
    
    if sp_mod is None: sp_tmp = None
    else:              sp_tmp = sp_mod.hfpc
        
    return np_mod.wf_symbs_string, np_mod.hfpc, sp_tmp

net1.wf.wf_symbs_string, net1.wf.hfpc_np, net1.wf.hfpc_sp = select_wf_model_quants(net1)

In [ ]:
net1.calc_wf_polarizations()

In [ ]:
# compute the WF polarizations and their derivatives
net1.calc_wf_polarizations_derivs_num()
# print(net1.hfp)

In [ ]:
# calculate fihser matrix, covariance matrix and errors 
# net.calc_errors()

# deriv_symbs_list = deriv_symbs_string.split(' ')
# deriv_hfp_list = ['del_' + ('log_' + item if item in conv_log else item) + '_hfp' for item in deriv_symbs_list]
# del_vs_f_dic1 = bfs.get_sub_dict(net1.del_hfpc,deriv_hfp_list,1)

# net1.fisher, net1.cov, net1.wc_fisher, net1.cond_num = fat.calc_fisher_cov_matrices(list(del_vs_f_dic1.values()), psd, f, 0)
# # net1.cov = LA.inv(net1.fisher + np.diag([0, PI ** (-2), 0, 0, 1, 1]))
# net1.errs = fat.get_errs_from_cov(net1.cov, net1.deriv_variables)

# # Calculate SNR
# import gwbench.snr as snr_mod

# net1.snr = snr_mod.snr_freq_array(net1.hfp, psd, f)

deriv_symbs_list = deriv_symbs_string.split(' ')
deriv_hfp_list = ['del_' + ('log_' + item if item in conv_log else item) + '_hfp' for item in deriv_symbs_list]
del_vs_f_dic1 = bfs.get_sub_dict(net1.del_hfpc,deriv_hfp_list,1)

net1.fisher, net1.cov, net1.wc_fisher, net1.cond_num = fat.calc_fisher_cov_matrices(list(del_vs_f_dic1.values()), psd, f, 0)
net1.errs = fat.get_errs_from_cov(net1.cov, net1.deriv_variables)
# print(net1.cov)


In [ ]:
#print error values

from math import floor, log10

def round_n(x, n):
    return round(x, n - int(floor(log10(abs(x)))) - 1)

# print the contents of the network objects
print("tc(ms): ", round_n(net1.errs['tc']*1000,4))
print("phic: ", round_n(net1.errs['phic'], 4))
print("log_Mch: ", round_n(net1.errs['log_Mc']*100,5)) # 100 here is for percentage error?
print("log_eta: ", round_n(net1.errs['log_eta']*100,4))
# print("log_M: ", round(net1.errs['log_M']*100,4))
# print("log_e0: ", round_n(net.errs['log_e0'],4))
# print()

# print("SNR: ", net.snr * np.sqrt(2/15) / np.sqrt(5/24))
# print("SNR: ", net1.snr)
# print("<SNR>: ", net1.snr/2.5)
# print("DL_Mpc: ", DL_Mpc)
# print("flso: ", flso)

In [ ]:
# print(net1.snr)
# print(net1.errs)

# print(net.fisher)
# print(net.cond_num)
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.loglog(net.f, np.abs(net.hfp))

# print(net.snr/10*500)

# print(net.fisher)
eye = np.identity(4)
# print((net.cov))
prod = (np.dot(net.fisher, net.cov) - eye)

print(np.max((prod)))

In [ ]:
# net2 = network.Network()
net2 = Network()
net2.wf = wfc.Waveform()

In [ ]:
# e0 = 0.1

inj_params2 = {
    'Mc':    Mc_solar,
    'eta':   0.2499,
    'chi1z': chi1zT,
    'chi2z': chi2zT,
    'DL':    DL_Mpc,
    'tc':    tc,
    'phic':  phic,
    'iota':  0,
    'ra':    0,
    'dec':   0.0,
    'psi':   0,
    'gmst0': 0
    }

# inj_params2 = {
#     'M':    M_solar * (1 + z),
#     'eta':   eta,
#     'chi1z': chi1z,
#     'chi2z': chi2z,
#     'DL':    500,
#     'tc':    0.,
#     'phic':  0,
#     'iota':  0.,
#     'ra':    0,
#     'dec':   np.pi/2,
#     'psi':   0,
#     'gmst0': 0,
#     'approx': approx,
#     'e0':    e0,
#     'amp_pn':    0, 
#     'phase_pn': pn,
#     'pn': pn
#     }


In [ ]:
net2.wf = wfc.Waveform()
net2.wf.wf_symbs_string, net2.wf.hfpc_np, net2.wf.hfpc_sp = select_wf_model_quants(net2)
net2.set_net_vars(
    f=f, inj_params=inj_params2,
    deriv_symbs_string=deriv_symbs_string,
    conv_cos=conv_cos, conv_log=conv_log,
    use_rot=use_rot
    )


In [ ]:
net2.calc_wf_polarizations_derivs_num()
del_vs_f_dic2 = bfs.get_sub_dict(net2.del_hfpc,deriv_hfp_list,1)
net2.fisher, net2.cov, net2.wc_fisher, net2.cond_num = fat.calc_fisher_cov_matrices(list(del_vs_f_dic2.values()), psd, f, 0)
net2.cov = LA.inv(net2.fisher + np.diag([0, 1 / PI ** 2, 0, 0, 1, 1]))

In [ ]:
net2.errs = fat.get_errs_from_cov(net2.cov, net2.deriv_variables)
net2.snr = snr_mod.snr_freq_array(net2.hfp, psd, f)

In [ ]:
del_hf1_keys = list(net1.del_hfpc.keys())
del_hf1_vals = list(net1.del_hfpc.values())

In [ ]:
del_hfp1 = {}
for i in range (len(del_hf1_keys)):
    if del_hf1_keys[i][-1:-4:-1] == 'pfh':
        #print(del_hf2_vals[i], del_hf2_keys[i])
        del_hfp1[del_hf1_keys[i]] = del_hf1_vals[i][1:]

del_hfp1_list = list(del_hfp1.values())

In [ ]:
from scipy.integrate import simps
def scalar_product_freq_array(integrand, freqs, df=None):
    if df is None:
        return np.real(simps(integrand,freqs))
    else:
        return

In [ ]:
f7_3 = scalar_product_freq_array(np.power(f,-7/3)/psd, f)
f7_3

In [ ]:
psi_1 = np.unwrap(np.angle(net1.hfp[1:]))
psi_2 = np.unwrap(np.angle(net2.hfp[1:]))


In [ ]:
def calc_sys_errs(psi_h, psi_g, del_hfp_list, hfp, rho, cov_mat, psd, f, deriv_symbs_list, conv_log):
    
    f7_3 = scalar_product_freq_array(np.power(f,-7/3)/psd, f)
    leng = len(deriv_symbs_list)
    sys_errs = np.zeros(leng)

    for i in range(leng):
        if deriv_symbs_list[i] in conv_log:
            deriv_symbs_list[i] = 'log_' + deriv_symbs_list[i]
        for j in range(leng):
            integ = -1j * np.power(f,-7/3)/psd * (psi_g - psi_h) * del_hfp_list[j] / hfp
            sys_errs[i] += rho**2  / f7_3 * cov_mat[i][j] * scalar_product_freq_array(integ, f)
    sys_errs = np.abs(sys_errs)
            
    print(sys_errs)
    sys_errs_dict = dict(zip(deriv_symbs_list, sys_errs))    
    return sys_errs_dict

In [ ]:
calc_sys_errs(psi_1, psi_2, del_hfp1_list, net1.hfp[1:], net1.snr, net1.cov, psd[1:], f[1:], deriv_symbs_string.split(), conv_log)

In [ ]:
net2.errs

In [ ]:
net1.errs